Model download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

In [ ]:
from imageai.Detection import ObjectDetection
import os

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

In [ ]:
detections_dict = dict()
dir_name = "images"
images_left = len(os.listdir(dir_name))

for filename in os.listdir(dir_name):
    detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename, output_type="array")[1]
    # detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename,
    #                                              output_image_path=f"{dir_name}/new_" + filename)

    detections_dict[filename] = []

    for eachObject in detections:
        detections_dict[filename].append((eachObject["name"], eachObject["percentage_probability"]))
        # print(filename, (eachObject["name"], eachObject["percentage_probability"]))

    images_left -= 1
    print("images left:", images_left)

In [ ]:
import pickle

with open("data/thumbnails_objects_dict.pkl", "wb") as f:
    pickle.dump(detections_dict, f)

In [ ]:
with open("data/thumbnails_objects_dict.pkl", "rb") as f:
    print(pickle.load(f))